[__<< Initial Data Exploration__](./01_Divvy_initial_data_exploration.ipynb) | [__Home__](../README.md) | [__Exploratory Data Analysis >>__](./03_Divvy_exploratory_data_analysis.ipynb)

# Divvy: Bike Sharing Forecast
## Data Cleaning

__Dataset:__ [Divvy Data](https://divvybikes.com/system-data) \
__Author:__ [Dmitry Luchkin](https://www.linkedin.com/in/dmitry-luchkin/) \
__Date:__ March 2025

### Goals/Objectives <a name="objectives"></a>

- Identify and address missing values (e.g., removal, imputation).
- Convert columns to appropriate data types (e.g., dates, categories).
- Identify and analyze outliers in the dataset.
- Identify and remove duplicate records.

### Notebooks <a class="anchor" name='notebooks'></a>

+ [Initial Data Exploration](./01_Divvy_data_exploration.ipynb)
+ __[Data Cleaning](./02_Divvy_data_cleaning.ipynb)__
+ [Exploratory Data Analysis](./03_Divvy_exploratory_data_analysis.ipynb)
+ [Feature Engineering](./04_Divvy_feature_engineering.ipynb)
+ [Modeling & Validation](./05_Divvy_modeling.ipynb)

### Import Libraries <a name='import-libraries'></a>

In [1]:
import datetime
import sys
import re
import pickle

import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')

sys.path.append('../02_scripts/')

%matplotlib inline

### Notebook Setup <a name='notebook-setup'></a>

In [2]:
# Pandas settings
pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 60
pd.options.display.float_format = '{:,.4f}'.format

# Visualization settings
from matplotlib import rcParams
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = (16, 5)   
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
rcParams['font.size'] = 12
rcParams['savefig.dpi'] = 300
plt.rc('xtick', labelsize=11)
plt.rc('ytick', labelsize=11)
%config InlineBackend.figure_format = 'retina'

In [3]:
from IPython.display import Markdown
from IPython.core.magic import register_cell_magic

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))

### ToDo's <a name='todos'></a>

In [4]:
# get all tasks from the previous phase

sys.path.append('../02_scripts/')
from todo_list import extract_todo_patterns

print(f'{'-'*5} TASKS FROM PREVIOUS PHASE {'-'*5}')
for todo in extract_todo_patterns('./01_Divvy_initial_data_exploration.ipynb'):
    print(f'TODO: {todo}')

----- TASKS FROM PREVIOUS PHASE -----
TODO: Analyze uniqueness of ride_id
TODO: Convert started_at and ended_at to datatime
TODO: Convert rideable_type and member_casual to category
TODO: Handle missing values in end_station_id, end_station_name, start_station_id, start_station_name, end_lat, end_lng


### Loading Data <a name='loading-data'></a>

In [5]:
# load stored data
%store -r data_divvy

In [6]:
data = data_divvy

In [7]:
# remove stored data
%store -d data_divvy

In [8]:
data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BD0A6FF6FFF9B921,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:40,Dearborn St & Erie St,110.0000,St. Clair St & Erie St,211.0000,41.8942,-87.6291,41.8944,-87.6234,casual
1,96A7A7A4BDE4F82D,electric_bike,2020-11-01 10:03:26,2020-11-01 10:14:45,Franklin St & Illinois St,672.0000,Noble St & Milwaukee Ave,29.0000,41.8910,-87.6353,41.9007,-87.6625,casual
2,C61526D06582BDC5,electric_bike,2020-11-01 00:34:05,2020-11-01 01:03:06,Lake Shore Dr & Monroe St,76.0000,Federal St & Polk St,41.0000,41.8810,-87.6168,41.8721,-87.6296,casual
3,E533E89C32080B9E,electric_bike,2020-11-01 00:45:16,2020-11-01 00:54:31,Leavitt St & Chicago Ave,659.0000,Stave St & Armitage Ave,185.0000,41.8955,-87.6820,41.9177,-87.6914,casual
4,1C9F4EF18C168C60,electric_bike,2020-11-01 15:43:25,2020-11-01 16:16:52,Buckingham Fountain,2.0000,Buckingham Fountain,2.0000,41.8765,-87.6204,41.8764,-87.6203,casual


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26384908 entries, 0 to 26384907
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 2.6+ GB


### Analizing of Variables <a name='analyzing-variables'></a>

#### `ride_id`

In [10]:
# TODO: Analyze uniqueness of ride_id
num_unique = data.groupby(['ride_id']).size()
print(f"Number of non unique ID: {len(num_unique[num_unique > 1])}")
num_unique[num_unique > 1][:5]

Number of non unique ID: 420


ride_id
011C8EF97AB0F30D    2
01406457A85B0AFF    2
021A73F8C18B932D    2
02606FBC7F8537EE    2
0334987B57662109    2
dtype: int64

In [11]:
data[data['ride_id'].isin(list(num_unique[num_unique > 1][:3].index))].sort_values(by='ride_id')

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
20601370,011C8EF97AB0F30D,classic_bike,2024-05-31 19:45:38.037,2024-06-01 20:45:33.862,Clifton Ave & Armitage Ave,TA1307000163,NaN,NaN,41.9182,-87.6569,NaN,NaN,casual
23403049,011C8EF97AB0F30D,classic_bike,2024-05-31 19:45:38,2024-06-01 20:45:33,Clifton Ave & Armitage Ave,TA1307000163,NaN,NaN,41.9182,-87.6569,NaN,NaN,casual
20417703,01406457A85B0AFF,electric_bike,2024-05-31 23:54:59.194,2024-06-01 00:01:47.626,NaN,NaN,Damen Ave & Chicago Ave,13132,41.8900,-87.6700,41.8958,-87.6772,member
23234868,01406457A85B0AFF,electric_bike,2024-05-31 23:54:59,2024-06-01 00:01:47,NaN,NaN,Damen Ave & Chicago Ave,13132,41.8900,-87.6700,41.8958,-87.6772,member
106076,021A73F8C18B932D,docked_bike,2020-11-25 16:35:39,2020-11-25 16:48:02,Clark St & Winnemac Ave,325.0000,Ravenswood Ave & Berteau Ave,314.0000,41.9733,-87.6679,41.9579,-87.6736,member
7169836,021A73F8C18B932D,docked_bike,2020-12-15 12:15:58,2020-11-25 16:48:02,Clark St & Winnemac Ave,TA1309000035,Ravenswood Ave & Berteau Ave,TA1309000018,41.9733,-87.6679,41.9579,-87.6736,member


In [12]:
data.duplicated(subset=['ride_id'], keep='last').sum()

420

In [13]:
# drop duplicated ride_id
data.drop_duplicates(subset=['ride_id'], keep='first', inplace=True)

In [14]:
assert data.duplicated(subset=['ride_id']).sum() == 0, "There are still duplicated ride_id's in the dataset."

#### `rideable_type`

In [15]:
data.groupby(['rideable_type']).size()

rideable_type
classic_bike        11354379
docked_bike          3534217
electric_bike       11351555
electric_scooter      144337
dtype: int64

In [16]:
# TODO: Convert rideable_type and member_casual to category
data['rideable_type'] = data['rideable_type'].astype('category')

In [17]:
assert isinstance(data['rideable_type'].dtype, pd.api.types.CategoricalDtype), '`rideable_type` variable is not a categorical type.'

#### `started_at` and `ended_at`

In [18]:
# TODO: Convert started_at and ended_at to datatime

In [19]:
data['started_at'] = pd.to_datetime(data['started_at'], format='mixed')
data['ended_at'] = pd.to_datetime(data['ended_at'], format='mixed')

In [20]:
assert pd.api.types.is_datetime64_dtype(data['started_at'])
assert pd.api.types.is_datetime64_dtype(data['ended_at'])

#### `start_station_id`, `start_station_name`, `end_station_id`, `end_station_name`, `end_lat` and `end_lng`

In [21]:
# TODO: Handle missing values in end_station_id, end_station_name, start_station_id, start_station_name, end_lat, end_lng

In [22]:
data[['start_station_id', 
      'start_station_name', 
      'end_station_id', 
      'end_station_name', 
      'end_lat', 
      'end_lng']].isna().groupby(data['rideable_type'], observed=True).sum()

,start_station_id,start_station_name,end_station_id,end_station_name,end_lat,end_lng
rideable_type,,,,,,
classic_bike,102,34,25681,25603,19816,19816
docked_bike,0,0,9135,9135,9249,9249
electric_bike,3501177,3500490,3672013,3671489,0,0
electric_scooter,67649,67649,70340,70340,0,0


According to the bike sharing service rules:
- __Electric bikes__ (_electric_bike_) can be parked both at Divvy dock stations and at one of over 600 public racks or posts.
- __Classic bikes__ (_classic_bike_) and docked bikes (_docked_bike_) must be returned to Divvy dock stations to finish the ride.

In [23]:
# usage range per bike type
data.groupby(['rideable_type'], observed=True)[['started_at', 'ended_at']].agg(['min', 'max']).map('{:%Y-%m-%d}'.format)

started_at                ended_at            
                         min         max         min         max
rideable_type                                                   
classic_bike      2020-12-02  2024-12-31  2020-12-02  2024-12-31
docked_bike       2020-01-01  2023-08-30  2020-01-01  2023-10-10
electric_bike     2020-07-13  2024-12-31  2020-07-13  2024-12-31
electric_scooter  2024-08-31  2024-09-30  2024-09-01  2024-09-30

The first date of usage of Electic Bike is __2020-07-13__ but Classic Bike - __2020-12-02__. The latest date of usage of a docked bike is __2023-10-10__, which could mean that there are no docked bikes available in the service anymore. As both classic bikes and docked bikes have to be docked at the end of the ride and both types are regular bikes, I have united these two categories into one for analysis and forecasting, naming the category "regular_bike".

In [24]:
data['bike_type'] = data['rideable_type']
data['bike_type'] = data['bike_type'].cat.add_categories(['regular_bike'])
data.loc[data['bike_type'].isin(['classic_bike', 'docked_bike']), 'bike_type'] = 'regular_bike'
data['bike_type'] = data['bike_type'].cat.remove_unused_categories()

In [25]:
data.groupby(['bike_type'], observed=True).size().map('{:_}'.format)

bike_type
electric_bike       11_351_555
electric_scooter       144_337
regular_bike        14_888_596
dtype: object

In [26]:
# percent of missing values per bike type
data[['start_station_id', 
      'start_station_name', 
      'end_station_id', 
      'end_station_name', 
      'end_lat', 
      'end_lng']].isna().groupby(data['bike_type'], observed=True).sum()/data.shape[0]*100

,start_station_id,start_station_name,end_station_id,end_station_name,end_lat,end_lng
bike_type,,,,,,
electric_bike,13.2698,13.2672,13.9173,13.9153,0.0000,0.0000
electric_scooter,0.2564,0.2564,0.2666,0.2666,0.0000,0.0000
regular_bike,0.0004,0.0001,0.1320,0.1317,0.1102,0.1102


The rides which were not finished correctly for the regular_bikes type constitute only 0.13% of the total count of rides - drop them.

In [27]:
data.drop(index=data[(data[['start_station_id', 
      'start_station_name', 'end_station_id', 'end_station_name', 'end_lat', 'end_lng']].isna().any(axis=1)) & \
          (data['bike_type'] == 'regular_bike')].index, inplace=True)

In [28]:
# repcent of missing values
data[['start_station_id', 
      'start_station_name', 
      'end_station_id', 
      'end_station_name', 
      'end_lat', 
      'end_lng']].isna().groupby(data['bike_type'], observed=True).sum()/data.shape[0]*100

,start_station_id,start_station_name,end_station_id,end_station_name,end_lat,end_lng
bike_type,,,,,,
electric_bike,13.2875,13.2849,13.9358,13.9338,0.0000,0.0000
electric_scooter,0.2567,0.2567,0.2670,0.2670,0.0000,0.0000
regular_bike,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### `member_casual`

In [29]:
data.groupby(['member_casual'], observed=True).size().map('{:_}'.format)

member_casual
casual    10_403_519
member    15_945_964
dtype: object

In [30]:
# TODO: Convert rideable_type and member_casual to category
data['member_casual'] = data['member_casual'].astype('category')

In [31]:
assert isinstance(data['member_casual'].dtype, pd.api.types.CategoricalDtype), '`member_casual` variable is not a categorical type.'

In [32]:
data.rename(columns={'member_casual': 'rider_type'}, inplace=True)

In [33]:
assert 'rider_type' in data.columns

In [34]:
data.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'rider_type', 'bike_type'],
      dtype='object')

### Drop redundant columns

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26349483 entries, 0 to 26384907
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  rider_type          category      
 13  bike_type           category      
dtypes: category(3), datetime64[ns](2), float64(4), object(5)
memory usage: 2.4+ GB


In [36]:
cols_to_drop = ['rideable_type']
data.drop(columns=cols_to_drop, axis=1, inplace=True)

In [37]:
# Verify columns dropped
assert all(col not in data.columns for col in cols_to_drop), "Columns have not been dropped"

In [38]:
data.columns

Index(['ride_id', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'rider_type', 'bike_type'],
      dtype='object')

### Duplicated Rows

In [39]:
assert data.duplicated().sum()==0

### Impute Missing Values <a name='impute-missing-values'></a>

In [40]:
data.isna().sum()[data.isna().sum() > 0]

start_station_name    3568139
start_station_id      3568826
end_station_name      3741829
end_station_id        3742353
dtype: int64

In [41]:
# percent of missing values
data[['start_station_id', 
      'start_station_name', 
      'end_station_id', 
      'end_station_name',]].isna().groupby(data['bike_type'], observed=True).sum()/data.shape[0]*100

,start_station_id,start_station_name,end_station_id,end_station_name
bike_type,,,,
electric_bike,13.2875,13.2849,13.9358,13.9338
electric_scooter,0.2567,0.2567,0.2670,0.2670
regular_bike,0.0000,0.0000,0.0000,0.0000


__Observations:__
- Only electric bikes have _NA_ values in `start_station_id`, `start_station_name`, `end_station_id`, and `end_station_name`. These columns do not need to be imputed as electric bike rides can be finished out of Divvy's dock stations.

### Outliers

Create additional variables to facilitate the identification and analysis of outliers and to enhance the [Exploratory Data Analysis (EDA)](./03_Divvy_exploratory_data_analysis.ipynb) process in the next stage.

In [42]:
dt = 'started_at'
data['year'] = data[dt].dt.year
data['quarter'] = data[dt].dt.quarter

# {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September',
#   10: 'October', 11: 'November', 12: 'December'}
data['month'] = data[dt].dt.month
data['day'] = data[dt].dt.day
data['hour'] = data[dt].dt.hour
data['minute'] = data[dt].dt.minute

# {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'}
data['season'] = data['month'] % 12 // 3 + 1 

# {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
data['weekday'] = data[dt].dt.weekday + 1

# {1: 'Late Night', 2: 'Early Morning', 3: 'Morning', 4: 'Noon', 5: 'Evening', 6: 'Night'}
data['part_day'] = (data['hour'] % 24 + 4) // 4 
# duration of ride in minutes
data['duration_min'] = data['ended_at'].sub(data['started_at']) / pd.Timedelta(minutes=1)

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26349483 entries, 0 to 26384907
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   started_at          datetime64[ns]
 2   ended_at            datetime64[ns]
 3   start_station_name  object        
 4   start_station_id    object        
 5   end_station_name    object        
 6   end_station_id      object        
 7   start_lat           float64       
 8   start_lng           float64       
 9   end_lat             float64       
 10  end_lng             float64       
 11  rider_type          category      
 12  bike_type           category      
 13  year                int32         
 14  quarter             int32         
 15  month               int32         
 16  day                 int32         
 17  hour                int32         
 18  minute              int32         
 19  season              int32         
 20  weekd

In [ ]:
# distance of ride in km
import geopy.distance

%time data['distance_km'] = data.apply(lambda x: geopy.distance.distance((x['start_lat'], x['start_lng']), \
                                                                         (x['end_lat'], x['end_lng'])).km, axis=1)

In [ ]:
# average speed
data['avg_speed_kmph'] = data['distance_km'] / data['duration_min'] * 60

In [ ]:
import calendar

# prepare categorical variables
# 'quarter', 'month', 'season', 'weekday', 'part_day'

data['quarter'] = pd.Categorical(data['quarter'], categories=list(range(1, 5)), ordered=True)
data['quarter'] = data['quarter'].cat.rename_categories({1: 'Q1', 
                                                         2: 'Q2', 
                                                         3: 'Q3', 
                                                         4: 'Q4'})

data['month'] = pd.Categorical(data['month'], categories=list(range(1, 13)), ordered=True)
data['month'] = data['month'].cat.rename_categories(dict(zip(range(1,13), list(calendar.month_name)[1:])))

data['season'] = pd.Categorical(data['season'], categories=list(range(1, 5)), ordered=True)
data['season'] = data['season'].cat.rename_categories({1: 'Winter', 
                                                       2: 'Spring', 
                                                       3: 'Summer', 
                                                       4: 'Autumn'})

data['weekday'] = pd.Categorical(data['weekday'], 
                                 categories=list(range(1, 8)), ordered=True)
data['weekday'] = data['weekday'].cat.rename_categories({1: 'Monday', 
                                                         2: 'Tuesday', 
                                                         3: 'Wednesday', 
                                                         4: 'Thursday', 
                                                         5: 'Friday', 
                                                         6: 'Saturday', 
                                                         7: 'Sunday'})

data['part_day'] = pd.Categorical(data['part_day'], categories=list(range(1, 7)), ordered=True)
data['part_day'] = data['part_day'].cat.rename_categories({1: 'Late Night', 
                                                         2: 'Early Morning', 
                                                         3: 'Morning',  
                                                         4: 'Noon',  
                                                         5: 'Evening',
                                                         6: 'Night'})

In [ ]:
data.info()

In [ ]:
data.describe().T

__Observations:__
- `duration_min` has negative values which we have to remove.
- `distance_km` has zero values which we have to analyze. Distance (km) can be 0 - start and end station are the same.
- `avg_speed_kmph` has negative and _inf_ values for bicycle. As the Divvy's ebikes have speed up to 20 MPH (~ 32.2 kmph), the trips with an average speed greater than __33 kmph__ will be considered outliers.

In [ ]:
print(f'Number of zero distance: {data[data['distance_km'] == 0.0].shape[0]:_}')
print(f'Zero distance: {data[data['distance_km'] == 0.0].shape[0]/data.shape[0]*100:.2f}%')

In [ ]:
print(f'Number of rides with undefined start and stop stations: ' \
        f'{data[(data['distance_km'] == 0.0) \
            & ((data['start_station_name'].isna()) \
             & (data['start_station_id'].isna()) \
              & (data['end_station_name'].isna()) \
               & (data['end_station_id'].isna()))].shape[0]:_}')

Remove the rides with zero distance and undefined start and stop station.

In [ ]:
data.drop(index=data[(data['distance_km'] == 0.0) \
                    & ((data['start_station_name'].isna()) \
                     & (data['start_station_id'].isna()) \
                      & (data['end_station_name'].isna()) \
                       & (data['end_station_id'].isna()))].index, inplace=True)

In [ ]:
# remove outliers in average speed
data = data[(data['avg_speed_kmph'] >= 0.0) & (data['avg_speed_kmph'] <= 33.0)]

In [ ]:
# remove duration which less 1 min.
data = data[(data['duration_min'] >= 1.0) & (data['distance_km'] >= 0.0)]

In [ ]:
data.shape

In [ ]:
data.describe().T

<span style="color:blue;">_# TODO: Use median instead of mean for `distance_km`, `duration_min` and `avg_speed_kmph`, such as median is more robust to outliers._</span>

### Store Cleaned Data

In [ ]:
# save processed dataset fot further analysis
data_divvy = data
%store data_divvy

In [ ]:
%load_ext watermark
%watermark -d -t -v -p numpy,pandas,matplotlib,seaborn,sklearn,statsmodels


---
\
[__<< Initial Data Exploration__](./01_Divvy_initial_data_exploration.ipynb) | [__Home__](../README.md) | [__Exploratory Data Analysis >>__](./03_Divvy_exploratory_data_analysis.ipynb)
\
\
Divvy: Bike Sharing Forecast, _August 2024_